In [ ]:
# Prevent TensorFlow from allocating all GPU memory

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
# Import necessary libraries and modules

import pandas as pd
import numpy as np
import os
from PIL import ImageFile
from tensorflow.keras import layers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import json
from tensorflow.keras.callbacks import BaseLogger
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback

In [ ]:
# Handle truncated images

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Define classes for benchmark model

classes = os.listdir('MOUNT_DIRECTORY/training')

In [ ]:
# Define data augmentation for training and validation sets (no augmentation for validation)

trainAug = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

valAug = ImageDataGenerator()

In [ ]:
# Define training and validatoin directories

train_dir = "MOUNT_DIRECTORY/training"
validation_dir = "MOUNT_DIRECTORY/validation"

# Use flow_from_directory to create training and validation generators, using batches of images

trainGen = trainAug.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=128)

valGen = valAug.flow_from_directory(
    validation_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
# Define callback to save model training progress after specified number of epochs

class EpochCheckpoint(Callback):
    def __init__(self, outputPath, every=5, startAt=0):
        super(Callback, self).__init__()
        self.outputPath = outputPath
        self.every = every
        self.intEpoch = startAt
  
    def on_epoch_end(self, epoch, logs={}):
        if (self.intEpoch + 1) % self.every == 0:
            p = os.path.sep.join([self.outputPath, "epoch_{}".format(self.intEpoch + 1)])
            self.model.save(p, overwrite = True)
            self.intEpoch += 1

In [ ]:
# Define callback to save and plot metrics during training 

class TrainingMonitor(BaseLogger):
    def __init__(self, figPath, jsonPath=None, startAt=0):
        # store the output path for the figure, the path to the JSON
        # serialized file, and the starting epoch
        super(TrainingMonitor, self).__init__()
        self.figPath = figPath
        self.jsonPath = jsonPath
        self.startAt = startAt

    def on_train_begin(self, logs={}):
        # initialize the history dictionary
        self.H = {}

        # if the JSON history path exists, load the training history
        if self.jsonPath is not None:
            if os.path.exists(self.jsonPath):
                self.H = json.loads(open(self.jsonPath).read())

                # check to see if a starting epoch was supplied
                if self.startAt > 0:
                    # loop over the entries in the history log and
                    # trim any entries that are past the starting
                    # epoch
                    for k in self.H.keys():
                        self.H[k] = self.H[k][:self.startAt]

    def on_epoch_end(self, epoch, logs={}):
        # loop over the logs and update the loss, accuracy, etc.
        # for the entire training process
        for (k, v) in logs.items():
            l = self.H.get(k, [])
            l.append(float(v))
            self.H[k] = l

        # check to see if the training history should be serialized
        # to file
        if self.jsonPath is not None:
            f = open(self.jsonPath, "w")
            f.write(json.dumps(self.H))
            f.close()

        # ensure at least two epochs have passed before plotting
        # (epoch starts at zero)
        if len(self.H["loss"]) > 1:
            # plot the training loss and accuracy
            N = np.arange(0, len(self.H["loss"]))
            plt.style.use("ggplot")
            plt.figure()
            plt.plot(N, self.H["loss"], label="train_loss")
            plt.plot(N, self.H["val_loss"], label="val_loss")
            plt.plot(N, self.H["accuracy"], label="train_acc")
            plt.plot(N, self.H["val_accuracy"], label="val_acc")
            plt.title("Training Loss and Accuracy [Epoch {}]".format(
                len(self.H["loss"])))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()

            # save the figure
            plt.savefig(self.figPath)
            plt.close()

In [ ]:
# Define preprocessing layer, followed by loading MobileNetV2 architecture with weights learned from ImageNet dataset
# Set include_top to false to remove final layers

i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
core = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False,
input_tensor=Input(shape=(224, 224, 3)))
# Freeze all layers
for layer in core.layers:
    layer.trainable = False
# Unfreeze final five layers
for layer in core.layers[150:]:
    layer.trainable = True

In [ ]:
# Add four additional layers to model, specifying output layer size using classes earlier defined

x = core(x)
baseModel = Model(inputs=[i], outputs=[x])

y = baseModel.output
y = layers.Flatten()(y)
y = layers.Dense(1024, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(len(classes), activation="softmax")(y)

model = Model(inputs=baseModel.input, outputs=y)

In [ ]:
# Set existing model to None to prevent loading any model and resuming training

existing_model = None

In [ ]:
# Compile model with predetermined hyperparameters

if existing_model is None:
    opt = Adam(learning_rate=1e-4)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy", "top_k_categorical_accuracy"])
# Provide the option of resuming from an existing model in case training is interrupted
else:
    model = load_model(existing_model)
    K.set_value(model.optimizer.learning_rate, 1e-5)

In [ ]:
# Define checkpoints and starting epoch to be used by callback when saving models and performance

checkpoints = "MOUNT_DIRECTORY/output/checkpoints"
start_epoch = 0

In [ ]:
# Define location for callbacks to save performance 

plotPath = os.path.sep.join(["MOUNT_DIRECTORY/output/", "benchmark.png"])
jsonPath = os.path.sep.join(["MOUNT_DIRECTORY/output/", "benchmark.json"])

In [ ]:
# Define callbacks so as to save model after every epoch

callbacks = [
    EpochCheckpoint(checkpoints, every=1,
        startAt=start_epoch),
    TrainingMonitor(plotPath,
        jsonPath=jsonPath,
        startAt=start_epoch)]

In [ ]:
# Train benchmark model

history = model.fit(
    trainGen,
    validation_data=valGen,
    epochs=50,
  callbacks=callbacks)

In [ ]:
# Define classes for family model

family_classes = os.listdir('MOUNT_DIRECTORY/family_data/training_family')

In [ ]:
# Clear existing training and validatoin generators

trainGen.reset()
valGen.reset()

In [ ]:
# Define new generators for the family model

train_dir = 'MOUNT_DIRECTORY/family_data/training_family'
validation_dir = "MOUNT_DIRECTORY/family_data/validation_family"

trainGen = trainAug.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=128)

valGen = valAug.flow_from_directory(
    validation_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
# Repeat model building process for family model, using family classes for output

i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
core = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False,
input_tensor=Input(shape=(224, 224, 3)))
for layer in core.layers:
    layer.trainable = False
for layer in core.layers[150:]:
    layer.trainable = True

x = core(x)
baseModel = Model(inputs=[i], outputs=[x])

y = baseModel.output
y = layers.Flatten()(y)
y = layers.Dense(1024, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(len(family_classes), activation="softmax")(y)

model = Model(inputs=baseModel.input, outputs=y)

In [ ]:
existing_model = None

In [ ]:
if existing_model is None:
    opt = Adam(learning_rate=1e-4, decay=1e-4 / 50)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy", "top_k_categorical_accuracy"])
else:
    model = load_model(existing_model)
    K.set_value(model.optimizer.learning_rate, 1e-5)

In [ ]:
checkpoints = "MOUNT_DIRECTORY/family_data/output/checkpoints"
start_epoch = 0

In [ ]:
plotPath = os.path.sep.join(["MOUNT_DIRECTORY/family_data/output/", "family.png"])
jsonPath = os.path.sep.join(["MOUNT_DIRECTORY/family_data/output/", "family.json"])

In [ ]:
# Train family model

history = model.fit(
    trainGen,
    validation_data=valGen,
    epochs=50,
  callbacks=callbacks)

In [ ]:
# Load benchmark model from best epoch

benchmark_model = load_model('MOUNT_DIRECTORY/output/checkpoints/epoch_48')

In [ ]:
# Define test generator (again without data augmentation) to evaluate model performance

test_dir = 'MOUNT_DIRECTORY/test'

testGen = valAug.flow_from_directory(
    test_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
# Evaluate bechmark model and save scores to csv file 

benchmark_score = benchmark_model.evaluate(testGen)
print("Evaluation finished")
benchmark_score_df = pd.DataFrame({'Loss':benchmark_score[0], 'Top-1-Accuracy':benchmark_score[1], 
                                   'Top-5-Accuracy':benchmark_score[2]}, index=['Performance'])
benchmark_score_df.to_csv('MOUNT_DIRECTORY/output/benchmark_score.csv')
benchmark_predIdxs = benchmark_model.predict(x=testGen)
print("Predictions finished")
benchmark_predIdxs = np.argmax(benchmark_predIdxs, axis=1)
# Use scikit-learn classificatoin report to create and save csv file of precision, recall and F1-score
benchmark_report = classification_report(testGen.classes, benchmark_predIdxs,
                                         target_names=testGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(benchmark_report).transpose()
df.to_csv('MOUNT_DIRECTORY/output/benchmark_evaluation.csv')

In [ ]:
# Create (and save) dataframe of incorrect predictions along with file names 

benchmark_mistakes = pd.DataFrame.from_dict({"Prediction": benchmark_predIdxs, "Actual": testGen.classes})
benchmark_mistakes.to_csv('MOUNT_DIRECTORY/output/benchmark_mistakes.csv')

In [ ]:
# Repeat process with out-of-sample data

oos_dir = 'MOUNT_DIRECTORY/out_of_sample'

oosGen = valAug.flow_from_directory(
    oos_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
benchmark_oos_score = benchmark_model.evaluate(oosGen)
print("Evaluation finished")
benchmark_oos_score_df = pd.DataFrame({'Loss':benchmark_oos_score[0], 'Top-1-Accuracy':benchmark_oos_score[1], 
                                       'Top-5-Accuracy':benchmark_oos_score[2]}, index=['Performance'])
benchmark_oos_score_df.to_csv('MOUNT_DIRECTORY/output/benchmark_oos_score.csv')
benchmark_oos_predIdxs = benchmark_model.predict(x=oosGen)
print("Predictions finished")
benchmark_oos_predIdxs = np.argmax(benchmark_oos_predIdxs, axis=1)
# Necessary to define labels, otherwise dataframe dimensions do not match
benchmark_oos_report = classification_report(oosGen.classes, benchmark_oos_predIdxs,
                                labels = list(range(0,79)), target_names=oosGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(benchmark_oos_report).transpose()
df.to_csv('MOUNT_DIRECTORY/output/benchmark_oos_evaluation.csv')

In [ ]:
# Save out-of-sample mistakes

benchmark_oos_mistakes = pd.DataFrame.from_dict({"Prediction": benchmark_oos_predIdxs, "Actual": oosGen.classes})
benchmark_oos_mistakes.to_csv('MOUNT_DIRECTORY/output/benchmark_oos_mistakes.csv')

In [ ]:
# Repeat process with family model

family_model = load_model('MOUNT_DIRECTORY/family_data/output/checkpoints/epoch_45')

In [ ]:
testGen.reset()

test_dir = 'MOUNT_DIRECTORY/family_data/test_family'

testGen = valAug.flow_from_directory(
    test_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
family_score = family_model.evaluate(testGen)
print("Evaluation finished")
score_df = pd.DataFrame({'Loss':family_score[0], 'Top-1-Accuracy':family_score[1], 'Top-5-Accuracy':family_score[2]}, 
                        index=['Performance'])
score_df.to_csv('MOUNT_DIRECTORY/family_data/output/score.csv')
family_predIdxs = family_model.predict(x=testGen)
print("Predictions finished")
family_predIdxs = np.argmax(family_predIdxs, axis=1)
report = classification_report(testGen.classes, family_predIdxs, target_names=testGen.class_indices.keys(), 
                               output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_csv('MOUNT_DIRECTORY/family_data/output/evaluation.csv')

In [ ]:
# Define felinae classes in order to build felinae model

felinae_classes = os.listdir('MOUNT_DIRECTORY/felinae/training')

In [ ]:
# Repeat process to create generators for felinae model

trainGen.reset()
valGen.reset()

train_dir = 'MOUNT_DIRECTORY/felinae/training'
validation_dir = "MOUNT_DIRECTORY/felinae/validation"

trainGen = trainAug.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=128)

valGen = valAug.flow_from_directory(
    validation_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
# Load family model (instead of pretrained MobileNetV2)

model = load_model('MOUNT_DIRECTORY/family_data/output/checkpoints/epoch_45')

In [ ]:
# Replace last layer using felinae classes

felinae_model= Model(inputs=model.input, outputs=model.layers[-2].output)
y = felinae_model.output
y = layers.Dense(len(felinae_classes), activation="softmax")(y)
felinae_model = Model(inputs=felinae_model.input, outputs=y)

In [ ]:
# Repeat training process

existing_model = None

In [ ]:
if existing_model is None:
    opt = Adam(learning_rate=1e-4, decay=1e-4 / 50)
    felinae_model.compile(loss="categorical_crossentropy", optimizer=opt, 
                          metrics=["accuracy", "top_k_categorical_accuracy"])
else:
    model = load_model(existing_model)
    K.set_value(model.optimizer.learning_rate, 1e-5)

In [ ]:
checkpoints = "MOUNT_DIRECTORY/felinae/output/checkpoints"
start_epoch = 0

In [ ]:
plotPath = os.path.sep.join(["MOUNT_DIRECTORY/felinae/output/", "felinae.png"])
jsonPath = os.path.sep.join(["MOUNT_DIRECTORY/felinae/output/", "feliane.json"])

In [ ]:
callbacks = [
    EpochCheckpoint(checkpoints, every=1,
        startAt=start_epoch),
    TrainingMonitor(plotPath,
        jsonPath=jsonPath,
        startAt=start_epoch)]

In [ ]:
history = felinae_model.fit(
    trainGen,
    validation_data=valGen,
    epochs=50,
    callbacks=callbacks)

In [ ]:
# Repeat process for canidae and sciuridae models

canidae_classes = os.listdir('MOUNT_DIRECTORY/canidae/training')

In [ ]:
trainGen.reset()
valGen.reset()

train_dir = 'MOUNT_DIRECTORY/canidae/training'
validation_dir = "MOUNT_DIRECTORY/canidae/validation"

trainGen = trainAug.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=128)

valGen = valAug.flow_from_directory(
    validation_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
model = load_model('MOUNT_DIRECTORY/family_data/output/checkpoints/epoch_45')
canidae_model= Model(inputs=model.input, outputs=model.layers[-2].output)
y = canidae_model.output
y = layers.Dense(len(canidae_classes), activation="softmax")(y)
canidae_model = Model(inputs=canidae_model.input, outputs=y)

In [ ]:
existing_model = None

In [ ]:
if existing_model is None:
    opt = Adam(learning_rate=1e-4, decay=1e-4 / 50)
    canidae_model.compile(loss="categorical_crossentropy", optimizer=opt, 
                          metrics=["accuracy", "top_k_categorical_accuracy"])
else:
    canidae_model = load_model(existing_model)
    K.set_value(canidae_model.optimizer.learning_rate, 1e-5)

In [ ]:
checkpoints = "MOUNT_DIRECTORY/canidae/output/checkpoints"
start_epoch = 0

In [ ]:
plotPath = os.path.sep.join(["MOUNT_DIRECTORY/canidae/output/", "canidae.png"])
jsonPath = os.path.sep.join(["MOUNT_DIRECTORY/canidae/output/", "canidae.json"])

In [ ]:
callbacks = [
    EpochCheckpoint(checkpoints, every=1,
        startAt=start_epoch),
    TrainingMonitor(plotPath,
        jsonPath=jsonPath,
        startAt=start_epoch)]

In [ ]:
history = canidae_model.fit(
    trainGen,
    validation_data=valGen,
    epochs=50,
    callbacks=callbacks)

In [ ]:
sciuridae_classes = os.listdir('MOUNT_DIRECTORY/sciuridae/training')

In [ ]:
trainGen.reset()
valGen.reset()

train_dir = 'MOUNT_DIRECTORY/sciuridae/training'
validation_dir = "MOUNT_DIRECTORY/sciuridae/validation"

trainGen = trainAug.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=128)

valGen = valAug.flow_from_directory(
    validation_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
model = load_model('MOUNT_DIRECTORY/family_data/output/checkpoints/epoch_45')

In [ ]:
sciuridae_model= Model(inputs=model.input, outputs=model.layers[-2].output)
y = sciuridae_model.output
y = layers.Dense(len(sciuridae_classes), activation="softmax")(y)
sciuridae_model = Model(inputs=sciuridae_model.input, outputs=y)

In [ ]:
existing_model = None

In [ ]:
if existing_model is None:
    opt = Adam(learning_rate=1e-4, decay=1e-4 / 50)
    sciuridae_model.compile(loss="categorical_crossentropy", optimizer=opt, 
                            metrics=["accuracy", "top_k_categorical_accuracy"])
else:
    sciuridae_model = load_model(existing_model)
    K.set_value(sciuridae_model.optimizer.learning_rate, 1e-5)

In [ ]:
checkpoints = "MOUNT_DIRECTORY/sciuridae/output/checkpoints"
start_epoch = 0

In [ ]:
plotPath = os.path.sep.join(["MOUNT_DIRECTORY/sciuridae/output/", "sciuridae.png"])
jsonPath = os.path.sep.join(["MOUNT_DIRECTORY/sciuridae/output/", "sciuridae.json"])

In [ ]:
callbacks = [
    EpochCheckpoint(checkpoints, every=1,
        startAt=start_epoch),
    TrainingMonitor(plotPath,
        jsonPath=jsonPath,
        startAt=start_epoch)]

In [ ]:
history = sciuridae_model.fit(
    trainGen,
    validation_data=valGen,
    epochs=50,
    callbacks=callbacks)

In [ ]:
# Load felinae model from best epoch, then repeat model evaluation process

felinae_model = load_model('MOUNT_DIRECTORY/felinae/output/checkpoints/epoch_40')

In [ ]:
testGen.reset()

felinae_test_dir = 'MOUNT_DIRECTORY/felinae/test'

felinae_testGen = valAug.flow_from_directory(
    felinae_test_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
felinae_score = felinae_model.evaluate(felinae_testGen)
print("Evaluation finished")
felinae_score_df = pd.DataFrame({'Loss':felinae_score[0], 'Top-1-Accuracy':felinae_score[1], 
                                 'Top-5-Accuracy':felinae_score[2]}, index=['Performance'])
felinae_score_df.to_csv('MOUNT_DIRECTORY/felinae/output/felinae_score.csv')
felinae_predIdxs = felinae_model.predict(x=felinae_testGen)
print("Predictions finished")
felinae_predIdxs = np.argmax(felinae_predIdxs, axis=1)
felinae_report = classification_report(felinae_testGen.classes, felinae_predIdxs,
                                       target_names=felinae_testGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(felinae_report).transpose()
df.to_csv('MOUNT_DIRECTORY/felinae/output/felinae_evaluation.csv')

In [ ]:
felinae_mistakes = pd.DataFrame.from_dict({"Prediction": felinae_predIdxs, "Actual": felinae_testGen.classes})
felinae_mistakes.to_csv('MOUNT_DIRECTORY/felinae/output/felinae_mistakes.csv')

In [ ]:
felinae_oos_dir = 'MOUNT_DIRECTORY/felinae/out_of_sample'

felinae_oosGen = valAug.flow_from_directory(
    felinae_oos_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
felinae_oos_score = felinae_model.evaluate(felinae_oosGen)
print("Evaluation finished")
felinae_oos_score_df = pd.DataFrame({'Loss':felinae_oos_score[0], 'Top-1-Accuracy':felinae_oos_score[1], 
                                     'Top-5-Accuracy':felinae_oos_score[2]}, index=['Performance'])
felinae_oos_score_df.to_csv('MOUNT_DIRECTORY/felinae/output/felinae_oos_score.csv')
felinae_oos_predIdxs = felinae_model.predict(x=felinae_oosGen)
print("Predictions finished")
felinae_oos_predIdxs = np.argmax(felinae_oos_predIdxs, axis=1)
felinae_oos_report = classification_report(felinae_oosGen.classes, felinae_oos_predIdxs,
                            labels = (0,1,2,3,4,5), target_names=felinae_oosGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(felinae_oos_report).transpose()
df.to_csv('MOUNT_DIRECTORY/felinae/output/felinae_oos_evaluation.csv')

In [ ]:
felinae_oos_mistakes = pd.DataFrame.from_dict({"Prediction": felinae_oos_predIdxs, "Actual": felinae_oosGen.classes})
felinae_oos_mistakes.to_csv('MOUNT_DIRECTORY/felinae/output/felinae_oos_mistakes.csv')

In [ ]:
# Repeat for Canidae and Sciuridae models

canidae_model = load_model('MOUNT_DIRECTORY/canidae/output/checkpoints/epoch_43')

In [ ]:
testGen.reset()

canidae_test_dir = 'MOUNT_DIRECTORY/canidae/test'

canidae_testGen = valAug.flow_from_directory(
    canidae_test_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
canidae_score = canidae_model.evaluate(canidae_testGen)
print("Evaluation finished")
canidae_score_df = pd.DataFrame({'Loss':canidae_score[0], 'Top-1-Accuracy':canidae_score[1], 
                                 'Top-5-Accuracy':canidae_score[2]}, index=['Performance'])
canidae_score_df.to_csv('MOUNT_DIRECTORY/canidae/output/canidae_score.csv')
canidae_predIdxs = canidae_model.predict(x=canidae_testGen)
print("Predictions finished")
canidae_predIdxs = np.argmax(canidae_predIdxs, axis=1)
canidae_report = classification_report(canidae_testGen.classes, canidae_predIdxs,
                                       target_names=canidae_testGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(canidae_report).transpose()
df.to_csv('MOUNT_DIRECTORY/canidae/output/canidae_evaluation.csv')

In [ ]:
canidae_mistakes = pd.DataFrame.from_dict({"Prediction": canidae_predIdxs, "Actual": canidae_testGen.classes})
canidae_mistakes.to_csv('MOUNT_DIRECTORY/canidae/output/canidae_mistakes.csv')

In [ ]:
canidae_oos_dir = 'MOUNT_DIRECTORY/canidae/out_of_sample'

canidae_oosGen = valAug.flow_from_directory(
    canidae_oos_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
canidae_oos_score = canidae_model.evaluate(canidae_oosGen)
print("Evaluation finished")
canidae_oos_score_df = pd.DataFrame({'Loss':canidae_oos_score[0], 'Top-1-Accuracy':canidae_oos_score[1], 
                                     'Top-5-Accuracy':canidae_oos_score[2]}, index=['Performance'])
canidae_oos_score_df.to_csv('MOUNT_DIRECTORY/canidae/output/canidae_oos_score.csv')
canidae_oos_predIdxs = canidae_model.predict(x=canidae_oosGen)
print("Predictions finished")
canidae_oos_predIdxs = np.argmax(canidae_oos_predIdxs, axis=1)
canidae_oos_report = classification_report(canidae_oosGen.classes, canidae_oos_predIdxs,
                            labels = (0,1,2,3,4,5), target_names=canidae_oosGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(canidae_oos_report).transpose()
df.to_csv('MOUNT_DIRECTORY/canidae/output/canidae_oos_evaluation.csv')

In [ ]:
canidae_oos_mistakes = pd.DataFrame.from_dict({"Prediction": canidae_oos_predIdxs, "Actual": canidae_oosGen.classes})
canidae_oos_mistakes.to_csv('MOUNT_DIRECTORY/canidae/output/canidae_oos_mistakes.csv')

In [ ]:
sciuridae_model = load_model('MOUNT_DIRECTORY/sciuridae/output/checkpoints/epoch_35')

In [ ]:
testGen.reset()

sciuridae_test_dir = 'MOUNT_DIRECTORY/sciuridae/test'

sciuridae_testGen = valAug.flow_from_directory(
    sciuridae_test_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
sciuridae_score = sciuridae_model.evaluate(sciuridae_testGen)
print("Evaluation finished")
sciuridae_score_df = pd.DataFrame({'Loss':sciuridae_score[0], 'Top-1-Accuracy':sciuridae_score[1], 
                                   'Top-5-Accuracy':sciuridae_score[2]}, index=['Performance'])
sciuridae_score_df.to_csv('MOUNT_DIRECTORY/sciuridae/output/sciuridae_score.csv')
sciuridae_predIdxs = sciuridae_model.predict(x=sciuridae_testGen)
print("Predictions finished")
sciuridae_predIdxs = np.argmax(sciuridae_predIdxs, axis=1)
sciuridae_report = classification_report(sciuridae_testGen.classes, sciuridae_predIdxs,
                                         target_names=sciuridae_testGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(sciuridae_report).transpose()
df.to_csv('MOUNT_DIRECTORY/sciuridae/output/sciuridae_evaluation.csv')

In [ ]:
sciuridae_mistakes = pd.DataFrame.from_dict({"Prediction": sciuridae_predIdxs, "Actual": sciuridae_testGen.classes})
sciuridae_mistakes.to_csv('MOUNT_DIRECTORY/sciuridae/output/sciuridae_mistakes.csv')

In [ ]:
sciuridae_oos_dir = 'MOUNT_DIRECTORY/sciuridae/out_of_sample'

sciuridae_oosGen = valAug.flow_from_directory(
    sciuridae_oos_dir,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=128)

In [ ]:
sciuridae_oos_score = sciuridae_model.evaluate(sciuridae_oosGen)
print("Evaluation finished")
sciuridae_oos_score_df = pd.DataFrame({'Loss':sciuridae_oos_score[0], 'Top-1-Accuracy':sciuridae_oos_score[1], 
                                       'Top-5-Accuracy':sciuridae_oos_score[2]}, index=['Performance'])
sciuridae_oos_score_df.to_csv('MOUNT_DIRECTORY/sciuridae/output/sciuridae_oos_score.csv')
sciuridae_oos_predIdxs = sciuridae_model.predict(x=sciuridae_oosGen)
print("Predictions finished")
sciuridae_oos_predIdxs = np.argmax(sciuridae_oos_predIdxs, axis=1)
sciuridae_oos_report = classification_report(sciuridae_oosGen.classes, sciuridae_oos_predIdxs, 
                        labels = (0,1,2,3,4,5), target_names=sciuridae_oosGen.class_indices.keys(), output_dict=True)
df = pd.DataFrame(sciuridae_oos_report).transpose()
df.to_csv('MOUNT_DIRECTORY/sciuridae/output/sciuridae_oos_evaluation.csv')

In [ ]:
sciuridae_oos_mistakes = pd.DataFrame.from_dict({"Prediction": sciuridae_oos_predIdxs, 
                                                 "Actual": sciuridae_oosGen.classes})
sciuridae_oos_mistakes.to_csv('MOUNT_DIRECTORY/sciuridae/output/sciuridae_oos_mistakes.csv')